In [ ]:
import os, gc, ast, cv2, glob, json,shutil, random, collections, \
       imageio, pydicom, numpy as np, pandas as pd, SimpleITK as sitk

from pathlib import Path
from tqdm.auto import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
def resampleDICOM(image, ref_image):
  resampler = sitk.ResampleImageFilter()
  resampler.SetReferenceImage(ref_image)
  resampler.SetInterpolator(sitk.sitkLinear)
  resampler.SetTransform(sitk.AffineTransform(image.GetDimension()))
  resampler.SetOutputSpacing(ref_image.GetSpacing())
  resampler.SetSize(ref_image.GetSize())
  resampler.SetOutputDirection(ref_image.GetDirection())
  resampler.SetOutputOrigin(ref_image.GetOrigin())
  resampler.SetDefaultPixelValue(image.GetPixelIDValue())
  resampled_image = resampler.Execute(image)
  return resampled_image

In [ ]:
def uint8_convert(arr):
  '''https://www.moad.computer/blog/handling-dicom-x-ray-images'''  
  arr = arr - np.min(arr)
  if np.max(arr) != 0:
    arr = arr / np.max(arr)
  return (arr * 255).astype(np.uint8)

In [ ]:
ROOT_DIR = '/kaggle/input'
DATA_DIR = f'{ROOT_DIR}/rsna-miccai-brain-tumor-radiogenomic-classification'
df = pd.read_csv(f'{DATA_DIR}/train_labels.csv'); print(df.shape)
train_path = f'{DATA_DIR}/train'
modalities= ['FLAIR', 'T1w', 'T1wCE', 'T2w']
ref_dir = f'{DATA_DIR}/train/00143/T1w'

In [ ]:
print(len(os.listdir(train_path)), df.BraTS21ID.nunique())

In [ ]:
def mkdir(dir_path):
  Path(dir_path).mkdir(parents=True, exist_ok=True)

In [ ]:
def save_resampled_dicom(index, dicom_path, data, data_type='train', progress_bar=None,
                         modalities=['FLAIR'], ref_dir='./', out_dir='./'):
  reader = sitk.ImageSeriesReader()
  reader.LoadPrivateTagsOn()
    
  ref_DICOM = reader.GetGDCMSeriesFileNames(f'{ref_dir}')
  reader.SetFileNames(ref_DICOM)
  ref_sitk = reader.Execute()

  patient_id  = str(data['BraTS21ID'][index]).zfill(5)
  dicom_paths = f"{dicom_path}/{patient_id}/"
  
  for m, t in enumerate(modalities):
    scan_dir = os.path.join(dicom_paths, t)
    if progress_bar is not None:
      progress_bar.set_description(f'Processing scan directory: {scan_dir} ...', 
                                   refresh=True)
    output_dir = os.path.join(out_dir, data_type, patient_id, t)
    mkdir(output_dir) 
    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{scan_dir}')
    reader.SetFileNames(filenamesDICOM)
    scan_sitk = reader.Execute()
    scan_resampled = resampleDICOM(scan_sitk, ref_sitk)
    scan_sitk_array = uint8_convert(sitk.GetArrayFromImage(scan_resampled))

    for j in range(len(scan_sitk_array[:,0,0])):
      imageio.imsave(output_dir+'/Image-'+str(j)+'.png', scan_sitk_array[j,:,:])
  
  if progress_bar is not None:
    progress_bar.set_description('Finished processing!!!', refresh=True)

In [ ]:
p = tqdm(range(len(df)))
for i in p:
  save_resampled_dicom(i, train_path, df, data_type='train', progress_bar=p,
                       modalities=modalities, ref_dir=ref_dir, out_dir='./')

In [ ]:
!rm ./*.ipynb
!zip -r -q ./resampledDICOM.zip ./
!rm -r ./train